In [7]:
### IMPORTS ###
import scipy.sparse
import scipy.stats
import numpy as np
import matplotlib.pyplot as pyplot
import sklearn.metrics as mtc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from pyemd import emd
import valores
import pickle


In [8]:
## FUNCIONES ##


class TFM:

    def __init__(self,path):
        self.X_tr = scipy.sparse.load_npz(path+'X_tr.npz')
        self.X_tst = scipy.sparse.load_npz(path+'X_tst.npz')
        self.y_tr = np.load(path+'y_tr.npy')
        self.y_tst = np.load(path+'y_tst.npy')
        # Se binarizan las caracteristicas 
        self.y_tr=MultiLabelBinarizer().fit_transform(self.y_tr)
        self.y_tst=MultiLabelBinarizer().fit_transform(self.y_tst)
        # Escalado 
        scaler = MaxAbsScaler().fit(self.X_tr)
        self.X_tr=scaler.transform(self.X_tr)
        self.X_tst=scaler.transform(self.X_tst)
        self.X_tr_simpli= None
        self.simplificador = False
        self.modelo = None
        self.clf= None
        self.columns= None
        print ('Dataset cargado y preprocesado')

    def reduce_features(self,num_data):
        columns=np.array([])
        for i in range(np.shape(self.X_tr)[1]):
            if self.X_tr[:,i].count_nonzero()> num_data:
                columns=np.append(columns,i)
        np.savetxt('features'+num_data+'.txt',columns,delimiter=',')
        print('Las características han sido reducidas')

    def simplify_dataset(self,features,tr_size):
        self.X_tr_simpli=self.X_tr
        self.y_tr_simpli=self.y_tr
        
        self.columns=np.loadtxt('features'+features+'.txt',delimiter=',')
        self.X_tr_simpli=self.X_tr[:,self.columns]
        """
        self.svd = TruncatedSVD(n_components=int(features), n_iter=5, random_state=42)
        
        self.X_tr_simpli=self.svd.fit_transform(self.X_tr)
        """
        self.X_tr_simpli, __, self.y_tr_simpli, __ = train_test_split(self.X_tr_simpli, self.y_tr_simpli, test_size=(1-tr_size), random_state=42)
        print('Simplificación finalizada')
        print(np.shape(self.X_tr_simpli))

    def use_simplify(self,boolean):
        self.simplificador= boolean
        
    def grid_search_cv(self,classifier,parameters,metodo):
        if (metodo == 0):
            classif = OneVsRestClassifier(classifier)
        elif (metodo==1):
            classif = ClassifierChain(classifier)
        model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4,verbose=50, n_jobs=-1,scoring='accuracy')
        if (self.simplificador):
            model_tunning.fit(self.X_tr_simpli, self.y_tr_simpli)
        else :
            model_tunning.fit(self.X_tr, self.y_tr)
        print (model_tunning.best_score_)
        print (model_tunning.best_params_)
        self.modelo = model_tunning.best_estimator_

    def fitting_classifier(self):
        if (self.simplificador):
            self.clf=self.modelo.fit(self.X_tr,self.y_tr)
        else :
            self.clf=self.modelo

    def metrics(self):
        if (self.simplificador):
            self.y_pred=self.clf.predict(self.X_tst)
        else :
            self.y_pred=self.clf.predict(self.X_tst)
        accuracy=mtc.accuracy_score(self.y_tst,self.y_pred)
        hamming=mtc.hamming_loss(self.y_tst,self.y_pred)
        precision=mtc.precision_score(self.y_tst,self.y_pred,average='micro')
        print("Total accuracy: ", accuracy)
        print("Hamming loss: ", hamming)
        print("Precision: ", precision)
        print(mtc.classification_report(self.y_tst,self.y_pred))
        #print("Accuracy per class:")
        aux=0
        for i in range(np.shape(self.y_pred)[1]):
            #print ("Class " ,i,": " ,mtc.accuracy_score(self.y_tst[:,i],y_pred[:,i]))
            aux=aux+mtc.accuracy_score(self.y_tst[:,i],self.y_pred[:,i])
        print("Accuracy media: ",aux/37)
        distance_matrix=np.ones((37,37))*(1/37)
        np.fill_diagonal(distance_matrix,0)
        emd_aux=0
        for i in range (np.shape(self.y_tst)[0]):
            emd_aux= emd_aux + emd(self.y_tst[i,:].astype(float),self.y_pred[i,:].astype(float),distance_matrix)
        
        print("EMD: ", emd_aux/(np.shape(self.y_tst)[0]))
        print("METRICAS DE LABEL RANKING: ")
        spearman=scipy.stats.spearmanr(self.y_tst, self.y_pred)[0]
        print("Spearmans Rank Correlation Coefficient: " + spearman)
        kendall=scipy.stats.kendalltau(self.y_tst, self.y_pred)[0]
        print("Kendall’s tau Correlation Coefficient: " + kendall)
        
        

class TFM_RBM:

    def __init__(self,path):
        self.X_tr = scipy.sparse.load_npz(path+'X_tr.npz')
        self.X_tst = scipy.sparse.load_npz(path+'X_tst.npz')
        self.y_tr = np.load(path+'y_tr.npy')
        self.y_tst = np.load(path+'y_tst.npy')
        # Se binarizan las caracteristicas 
        self.y_tr=MultiLabelBinarizer().fit_transform(self.y_tr)
        self.y_tst=MultiLabelBinarizer().fit_transform(self.y_tst)
        # Escalado 
        scaler = MaxAbsScaler().fit(self.X_tr)
        self.X_tr=scaler.transform(self.X_tr)
        self.X_tst=scaler.transform(self.X_tst)
        self.modelo = None
        self.clf= None
        self.columns= None
        print ('Dataset cargado y preprocesado')

    def rbm_extraction(self):
        rbm=BernoulliRBM(random_state=0, verbose=True,learning_rate=0.06,n_iter=20,n_components=200)
        rbm.fit(self.X_tr)
        filename = 'rbm_data_200.sav'
        pickle.dump(rbm, open(filename, 'wb'))
        print('Modelo RBM guardado')


        
    def rbm_gridsearch(self,classifier,metodo,filename):
        self.rbm_data = pickle.load(open(filename, 'rb'))
        self.X_tr=self.rbm_data.transform(self.X_tr)
        print ("Transformación finalizada")
        if (metodo == 0):
            classif = OneVsRestClassifier(classifier)
        elif (metodo==1):
            classif = ClassifierChain(classifier)
        model_tunning = GridSearchCV(classif, param_grid=parameters,cv=4,verbose=50, n_jobs=-1,scoring='accuracy')
        model_tunning.fit(self.X_tr, self.y_tr)
        print (model_tunning.best_score_)
        print (model_tunning.best_params_)
        self.modelo = model_tunning.best_estimator_

    def fitting_classifier(self):
        self.clf=self.modelo

    def metrics(self):
        self.y_pred=self.clf.predict(self.rbm_data.transform(self.X_tst))
        accuracy=mtc.accuracy_score(self.y_tst,self.y_pred)
        hamming=mtc.hamming_loss(self.y_tst,self.y_pred)
        precision=mtc.precision_score(self.y_tst,self.y_pred,average='micro')
        print("Total accuracy: ", accuracy)
        print("Hamming loss: ", hamming)
        print("Precision: ", precision)
        print(mtc.classification_report(self.y_tst,self.y_pred))
        #print("Accuracy per class:")
        aux=0
        for i in range(np.shape(self.y_pred)[1]):
            #print ("Class " ,i,": " ,mtc.accuracy_score(self.y_tst[:,i],y_pred[:,i]))
            aux=aux+mtc.accuracy_score(self.y_tst[:,i],self.y_pred[:,i])
        print("Accuracy media: ",aux/37)
        distance_matrix=np.ones((37,37))*(1/37)
        np.fill_diagonal(distance_matrix,0)
        emd_aux=0
        for i in range (np.shape(self.y_tst)[0]):
            emd_aux= emd_aux + emd(self.y_tst[i,:].astype(float),self.y_pred[i,:].astype(float),distance_matrix)
        
        print("EMD: ", emd_aux/(np.shape(self.y_tst)[0]))
        print("METRICAS DE LABEL RANKING: ")
        spearman=scipy.stats.spearmanr(self.y_tst, self.y_pred)[0]
        print("Spearmans Rank Correlation Coefficient: " + spearman)
        kendall=scipy.stats.kendalltau(self.y_tst, self.y_pred)[0]
        print("Kendall’s tau Correlation Coefficient: " + kendall)
        
        
        

In [ ]:
path_string=input('Introduce el directorio del dataset')
clasificacion=TFM(path_string)
print('Inicio del programa clasificador')

while True:
    
    ask2=int(input('Que clasificador vas a usar?: '))
    clasificador=valores.clasificadores_dict[ask2]
    metod=int(input('Que metodo vas a usar?: Introduce 0 para One vs Rest, o 1 para Classifier Chain'))
        
    while True:
        ask1=input(('Quieres usar simplificacion para entrenar el algoritmo?'))
        if (ask1.lower()=='si'):
            non_zero_data=input(('Introduce el numero de elementos no nulos por categoria que consideras aceptable'))
            training_size=float(input('Introduce el porcentaje de tamano de datos de entrenamiento que quieres usar para simplificar'))
            print('Realizando simplificacion del dataset...')
            clasificacion.simplify_dataset(non_zero_data,training_size)
            clasificacion.use_simplify(True)
        elif (ask1.lower()=='no'):
            clasificacion.use_simplify(False)
        print("Iniciando tunning de los parámetros:")
        clasificacion.grid_search_cv(clasificador,valores.parametros_dict[(clasificador,metod)],metod)    
        print("Tunning finalizado:")  
        ask5=input('Quieres continuar con el entrenamiento?')
        if (ask5.tolower()=='no'){
            break
        }
        print("Entrenando el algoritmo con los mejores parametros...")
        clasificacion.fitting_classifier()
        print("Entrenamiento finalizado, se procede a predecir las etiquetas")
        print("Mostrando metricas obtenidas")
        clasificacion.metrics()
        ask3=input('Quieres probar a variar la simplificacion?')
        if (ask3.lower()=='si'):
            continue
        elif (ask3.lower()=='no'):
            break
            
    ask4=input('Quieres probar con otro clasificador o finalizamos el programa?')
    if (ask4.lower()=='si'):
        continue
    elif (ask4.lower()=='no'):
        break

print("Fin del programa clasificador")
    



In [ ]:
path_string=input('Introduce el directorio del dataset')
clasificacion=TFM_RBM(path_string)
print('Inicio del programa clasificador')

while True:
    
    ask2=int(input('Que clasificador vas a usar?: '))
    clasificador=valores.clasificadores_dict[ask2]
    metod=int(input('Que metodo vas a usar?: Introduce 0 para One vs Rest, o 1 para Classifier Chain'))
        
    while True:
        print("Iniciando tunning de los parámetros:")
        clasificacion.pipeline_rbm(clasificador,metod)    
        print("Tunning finalizado:")  
        ask5=input('Quieres continuar con el entrenamiento?')
        print("Entrenando el algoritmo con los mejores parametros...")
        clasificacion.fitting_classifier()
        print("Entrenamiento finalizado, se procede a predecir las etiquetas")
        print("Mostrando metricas obtenidas")
        clasificacion.metrics()
        break;
            
    ask4=input('Quieres probar con otro clasificador o finalizamos el programa?')
    if (ask4.lower()=='si'):
        continue
    elif (ask4.lower()=='no'):
        break

print("Fin del programa clasificador")
    




In [ ]:
path_string="../../Datasets/dataset/"
metod=0
clasificador=valores.LogisticRegression()
parameters={
    "estimator__C": [1],
    "estimator__solver": ["liblinear"],
    "estimator__tol": [0.01]
}
clasificacion=TFM(path_string)
clasificacion.use_simplify(False)
clasificacion.grid_search_cv(clasificador,parameters,metod)
clasificacion.fitting_classifier()
clasificacion.metrics()

In [9]:
path_string="../../Datasets/dataset/"
clasificacion=TFM_RBM(path_string)
clasificacion.rbm_extraction()



Dataset cargado y preprocesado
[BernoulliRBM] Iteration 1, pseudo-likelihood = -33.43, time = 1411.94s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -32.95, time = 1380.50s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -32.73, time = 1384.05s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -32.49, time = 1380.87s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -31.79, time = 1380.62s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -33.60, time = 1380.58s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -31.99, time = 1382.60s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -32.43, time = 1383.53s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -33.59, time = 1380.42s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -31.74, time = 1381.17s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -33.76, time = 1380.38s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -32.45, time = 1381.22s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -32.08, time = 1384.51s
[BernoulliRBM] Iteration 14, 